In [1]:
#import modules
import tensorflow as tf
import time
import os
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#define args
LEARNING_RATE = 0.01
batch_size = 128
n_epochs = 1

DROPOUT = 0.75
N_CLASSES = 10
SKIP_STEP = 10

In [4]:
X = tf.placeholder(tf.float32, [None, 784], name="X_placeholder")
Y = tf.placeholder(tf.float32, [None, 10], name="Y_placeholder")

dropout = tf.placeholder(tf.float32, name='dropout')

In [5]:
global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

def paddingTwoZero(image):
    paddings = [[2, 2], [2, 2]]
    newImage = tf.pad(image, paddings, "CONSTANT")
    return newImage

with tf.variable_scope('conv1') as scope:
    temp_images = tf.reshape(X, shape=[-1, 28, 28, 1])
    images = tf.image.resize_images(temp_images, [32, 32])
    
    kernel = tf.get_variable('kernel', [5, 5, 1, 6], 
                        initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [6],
                        initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(images, kernel, strides=[1, 1, 1, 1], padding='VALID')
    conv1 = tf.nn.relu(conv + biases, name=scope.name)
#     output:28 * 28 * 6
    
with tf.variable_scope('pool1') as scope:
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                            padding='VALID')    
#     output:14 * 14 * 6

with tf.variable_scope('conv2') as scope:
    kernel = tf.get_variable('kernels', [5, 5, 6, 16], 
                        initializer=tf.truncated_normal_initializer())
    biases = tf.get_variable('biases', [16],
                        initializer=tf.random_normal_initializer())
    conv = tf.nn.conv2d(pool1, kernel, strides=[1, 1, 1, 1], padding='VALID')
    conv2 = tf.nn.relu(conv + biases, name=scope.name)
#     output:10 * 10 * 16

with tf.variable_scope('pool2') as scope:
    pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                            padding='VALID')
#     output:5 * 5 * 16


with tf.variable_scope('fc') as scope:
    input_features = 5 * 5 * 16
    w = tf.get_variable('weights', [input_features, 512],
                        initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [512],
                        initializer=tf.random_normal_initializer())

    pool2 = tf.reshape(pool2, [-1, input_features])
    fc = tf.nn.relu(tf.matmul(pool2, w) + b, name='relu')

    fc = tf.nn.dropout(fc, dropout, name='relu_dropout')

with tf.variable_scope('softmax_linear') as scope:
    w = tf.get_variable('weights', [512, N_CLASSES],
                        initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('biases', [N_CLASSES],
                        initializer=tf.random_normal_initializer())
    logits = tf.matmul(fc, w) + b


In [6]:
with tf.name_scope('loss'):
    entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)
    loss = tf.reduce_mean(entropy, name='loss')
    
optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss, 
                                        global_step=global_step)


In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    initial_step = global_step.eval()

    start_time = time.time()
    n_batches = int(mnist.train.num_examples / batch_size)

    total_loss = 0.0
    #train
    for index in range(initial_step, n_batches * n_epochs):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        _, loss_batch = sess.run([optimizer, loss], 
                                feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        total_loss += loss_batch
        if (index + 1) % SKIP_STEP == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
    
    print("Optimization Finished!")
    print("Total time: {0} seconds".format(time.time() - start_time))
    
    #test
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], 
                                        feed_dict={X: X_batch, Y:Y_batch, dropout: DROPOUT}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)   
    
    print("Accuracy {0}".format(total_correct_preds/mnist.test.num_examples))

Average loss at step 10: 12745.0
Average loss at step 20: 2417.7
Average loss at step 30: 826.1
Average loss at step 40: 351.6
Average loss at step 50: 218.1
Average loss at step 60: 198.8
Average loss at step 70: 162.3
Average loss at step 80: 153.6
Average loss at step 90: 108.2
Average loss at step 100: 108.9
Average loss at step 110:  68.3
Average loss at step 120:  71.0
Average loss at step 130:  65.1
Average loss at step 140:  63.7
Average loss at step 150:  59.6
Average loss at step 160:  59.7
Average loss at step 170:  53.5
Average loss at step 180:  46.6
Average loss at step 190:  42.7
Average loss at step 200:  47.7
Average loss at step 210:  49.5
Average loss at step 220:  40.1
Average loss at step 230:  32.1
Average loss at step 240:  35.3
Average loss at step 250:  28.7
Average loss at step 260:  35.9
Average loss at step 270:  31.5
Average loss at step 280:  32.5
Average loss at step 290:  29.4
Average loss at step 300:  28.3
Average loss at step 310:  25.6
Average loss a

In [11]:
# loss很低，但是准确率为什么也这么低